In [3]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
class LecturesData(object):
    '''
    获取讲座的信息
    '''
    COLUMNS = ['title', 'reporter', 'time', 'place', 'holder']
    def __init__(self):
        self.root_url = "https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/"
        self.root = requests.get(self.root_url)
        self.root_bs = BeautifulSoup(self.root.text, 'lxml')
        self.raw_columns = ['报告题目:', '报告人:', '报告时间:', '报告地点:', '主办单位:', '简介:', '链接:']
        self.columns = ['title', 'reporter', 'time', 'place', 'holder', 'abstract', 'url']
        self.col_dict = dict(zip(self.raw_columns, self.columns))
    
    def get_lectures_details(self, detail_url:str):
        """
        根据传入的讲座详情页url进行解析，提取出指定的列
        Parameter:
            detail_url: str, 详情网页的url

        Return:
            dict
        """
        detail_page=requests.get(detail_url)
        detail_bs = BeautifulSoup(detail_page.text, 'lxml')
        detail_trs = detail_bs.find_all('tr')
        detail_tr_dict = {}
        for tr in detail_trs:
            tds = tr.find_all('td')
            if len(tds) <= 0:
                continue
            node = tds[0].find('div')
            if node is not None and node.string is not None:
                s = node.string.strip()
                if s in self.raw_columns:
                    col = self.col_dict[s]
                    if col == 'abstract':
                        detail_tr_dict['abstract'] = ' '.join(tds[1].text.split())
                    else:
                        detail_tr_dict[col] = tds[1].string
        detail_tr_dict['url'] = detail_url
        return detail_tr_dict

    def get_lectures_tag(self, tag_node, page_count:int=-1):
        """
        根据传入的讲座tag的node进行解析
        Parameter:
            tag_node: bs4.element.Tag, 
            page_count: int, 读取的页数 

        Return:
            list of dict
        """
        tag_data_list=[]
        tag = tag_node.string
        tag_url = tag_node['href']
        tag_url = self.root_url + tag_url
        tag_page = requests.get(tag_url)
        count = 0
        while True:
            tag_bs = BeautifulSoup(tag_page.text, 'lxml')
            lec_trs = tag_bs.find_all('tr')
            ccc=0
            for tr in lec_trs:
                # if tr.get('bgcolor', None) is None or tr.get('class', None) is not None:
                #     continue
                tds = tr.find_all('td')
                if tds is None or len(tds) < 2:
                    continue
                a = tds[1].find('a')
                if a is None:
                    continue
                detail_url = a.get('href')
                if 'detail' not in detail_url:
                    continue
                detail_url = self.root_url + detail_url
                print(detail_url)
                tag_data_list.append(self.get_lectures_details(detail_url))
                # 测试用代码，限制读取条数减少用时
                # if len(tag_data_list) > 4:
                #     print(pd.DataFrame(tag_data_list))
                #     break
            nextpage_node=tag_bs.find('a', string=re.compile('.*下.*页.*'))
            if nextpage_node is None:
                break
            nextpage_url = self.root_url + nextpage_node['href']
            tag_page = requests.get(nextpage_url)
            count += 1
            if page_count > 0 and count >= page_count:
                break
        return tag_data_list

    def get_lectures_data(self, tag_count:int=-1, page_count:int=-1):
        """
        读取分tag的讲座信息
        Parameter:
            tag_count: int, 读取的tag数 
            page_count: int, 读取的页数 

        Return:
            dict of pd.DataFrame
        """
        root_trs = self.root_bs.find_all(name='td', id=re.compile("category[0-9][0-9][0-9][0-9].+"))
        data_dict = {}
        count = 0
        for tag_root in root_trs:
            tag_data_list = []
            tag_node = tag_root.find('a')
            if tag_node is None:
                break
            tag = tag_node.string
            data_dict[tag] = []
            data_dict[tag] = pd.DataFrame.from_records(self.get_lectures_tag(tag_node, page_count))
            count += 1
            if count >= tag_count:
                break
        return data_dict

In [23]:
lectureinfo = LecturesData()

In [24]:
result = lectureinfo.get_lectures_data(tag_count=2,page_count=2)

https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/detail.jsp?seq=20607&boardid=35&pageno=
https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/detail.jsp?seq=20613&boardid=35&pageno=
https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/detail.jsp?seq=20596&boardid=35&pageno=
https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/detail.jsp?seq=20582&boardid=35&pageno=
https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/detail.jsp?seq=20566&boardid=35&pageno=
                                               title       reporter  \
0                                       为学四问之一：“为何学”            谢维和   
1  国有企业党的建设工作研讨会----暨习近平总书记在全国国有企业党的建设工作会议发表重要讲话4...       李耀强、刘新刚等   
2                                 论坛预告｜计算器具：过去、现在与未来  第二届清华科学博物馆学论坛   
3                                后五四时期中国精神人文主义的建构性探索             李萍   
4                      讲座预告 | 中美风暴下的风险与对策-金融EMBA师友沙龙         鞠建东教授等   

               time          place                 holder  \
0  2020-10-22 14:00             待定                  教育研究院   
1  2020-10-11 08:30  

In [25]:
result

{'人文与社会科学':                                                title       reporter  \
 0                                       为学四问之一：“为何学”            谢维和   
 1  国有企业党的建设工作研讨会----暨习近平总书记在全国国有企业党的建设工作会议发表重要讲话4...       李耀强、刘新刚等   
 2                                 论坛预告｜计算器具：过去、现在与未来  第二届清华科学博物馆学论坛   
 3                                后五四时期中国精神人文主义的建构性探索             李萍   
 4                      讲座预告 | 中美风暴下的风险与对策-金融EMBA师友沙龙         鞠建东教授等   
 5                           执简驭繁，匠心天工——论晚商时期的青铜铸造与生产            刘煜    
 
                time          place                 holder  \
 0  2020-10-22 14:00             待定                  教育研究院   
 1  2020-10-11 08:30  清华大学廖凯原楼（法学院）        清华大学中国现代国有企业研究院   
 2  2020-09-24 11:00     蒙民伟人文楼B206  清华大学科学博物馆（筹）、清华大学科学史系   
 3  2020-09-18 15:30     清华大学四教4206                马克思主义学院   
 4  2020-09-12 20:00             线上          清华五道口金融EMBA中心   
 5  2019-11-15 15:00        人文楼B206               人文学院科学史系   
 
                                             ab

In [26]:
for key in result:
    result[key].to_csv("{}.csv".format(key), index=False)

In [27]:
result['经济学']

,title,reporter,time,place,holder,abstract,url
0,国有企业党的建设工作研讨会----暨习近平总书记在全国国有企业党的建设工作会议发表重要讲话4...,李耀强、刘新刚等,2020-10-11 08:30,清华大学廖凯原楼（法学院）,清华大学中国现代国有企业研究院,经管学院第202001号国有企业党的建设工作研讨会----暨习近平总书记在全国国有企业党的建...,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...
1,讲座预告 | 中美风暴下的风险与对策-金融EMBA师友沙龙,鞠建东教授等,2020-09-12 20:00,线上,清华五道口金融EMBA中心,2020年以来，美国总统大选临近，叠合新冠疫情、经济衰退和种族骚乱等影响，美国逐步升级了对中...,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...
2,2019中国金融科技学术年会,None,2019-12-21 08:30,北京香山饭店,None,为促进金融科技研究，推动中国金融科技的健康稳定和可持续发展，金融科技教育与研究50人论坛发起...,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...
3,名师微沙龙：文科PI系列第一场,仇军,2019-12-11 16:30,李文正图书馆负一层邺架轩书店,文科建设处,科学研究是否充满生命力，就看研究者是否有问题意识，是否能股发现并提出大量的问题。什么样的问题...,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...
4,清华五道口金融家大讲堂,Sheila M'Mbijjewe,2019-12-10 16:00,清华大学五道口金融学院1-200,清华大学五道口金融学院,主办单位：清华大学五道口金融学院 支持单位：渤海银行官方媒体：清华金融评论 未央网地 点：清...,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...
5,第6场【清华大学“闻道”国际论坛】欧洲复兴开发银行2018-2019年度转型报告，EBRD ...,（1）亚历山大·普列汉诺夫Alexandre Plekhanov,2019-03-05 19:00,"清华大学苏世民书院，B104教室，地下一层, B104, Schwarzman Colleg...",清华大学苏世民书院,报告人：（1）亚历山大•普列汉诺夫，欧开行高级经济学家（主讲）该年度报告的主题是“Work ...,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...


In [14]:
def parse_line(tr):
    cols=tr.find_all('td')
    cols[1]=cols[1].find('a')
    return [col.string for col in cols], cols[1]

In [28]:
root = "https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/"
result = requests.get(root)
bs_root = BeautifulSoup(result.text, 'lxml')
trs_root=bs_root.find_all(name='td', id=re.compile("category[0-9][0-9][0-9][0-9].+"))

In [29]:
cols=['报告题目:', '报告人:', '报告时间:', '报告地点:', '主办单位:']

In [30]:
tag_root = trs_root[0]

In [31]:
type(tag_root)

bs4.element.Tag

In [44]:
lec_node=tag_root.find('a')
lec_node.text

'人文与社会科学'

In [45]:
lec_url=lec_node['href']

In [46]:
tag_page=requests.get(root+lec_url)

In [47]:
bs_tag=BeautifulSoup(tag_page.text, 'lxml')
trs_lec=bs_tag.find_all('tr')

In [48]:
trs_lec[10]


<tr bgcolor="#FFFFFF">
<td align="center">2020-09-24 11:00 </td>
<td><a href="detail.jsp?seq=20596&amp;boardid=35&amp;pageno=" target="_blank">论坛预告｜计算器具：过去、现在与未来</a> </td>
<td>第二届清华科学博... </td>
<td>蒙民伟人文楼B206 </td>
</tr>

In [50]:
detail_url=trs_lec[10].find_all('td')[1].find('a')['href']

In [52]:
detail_page=requests.get(root+detail_url)
bs_detail=BeautifulSoup(detail_page.text, 'lxml')

In [53]:
trs_detail=bs_detail.find_all('tr')

In [65]:
trs_detail[12]

<tr><td align="right" width="41%"><span class="STYLE5"><font color="#4E73A2">to</font></span>  </td>
<td valign="startTime" width="59%">
<input class="in" id="endTime" name="endTime" size="12" type="text"/> <img align="absbottom" onclick="cal18.select(document.forms[0].endTime,'endTime','yyyy-MM-dd'); return false;" src="/images/calendar.png"/>
</td>
</tr>

In [73]:
res=[]
for tr in trs_detail:
    tds=tr.find_all('td')
    if len(tds)>0:
        node1=tds[0].find('div')
        if node1 is not None and node1.string is not None:
            res.append(tds)

In [91]:
''.join(res[-1][1].text.split())

'清华科学博物馆学论坛是以科学博物馆学和科学仪器史研究为主题的学术活动，由清华大学科学博物馆（筹）和清华大学科学史系主办。论坛以理论探讨为主，与展览实践密切结合，体现高校博物馆特色，促进高校人才培养与大众科学传播。论坛主题：“计算器具：过去、现在与未来”论坛时间：2020年9月24日11:00-17:30论坛地点：清华大学蒙民伟人文楼B206腾讯会议:577193312会议日程第1场11:00–12:00主持人：蒋澈（清华大学科学史系）11:00–11:30报告题目：《从算筹到算盘:中国传统算具与算法体系的特色》报告人：冯立昇（清华大学科技史暨古文献研究所）11:30–12:00报告题目：《“紧急措施”的由来与计算机事业的成效》报告人：徐祖哲（计算机发展史研究专家）第2场13:30–15:00主持人：王哲然（清华大学科学史系）13:30–14:00报告题目：《子木园里的“神机妙算”》报告人：谭丽娅（合肥子木园博物馆）14:00–14:30报告题目：《天河系列超级计算机应用创新发展》报告人：菅晓东（国家超级计算机天津中心）14:30–15:00报告题目：《追寻中国计算机创业者足迹》报告人：胡春明（中国计算机学会）第3场15:15–16:45主持人：胡翌霖（清华大学科学史系）15:15–15:45报告题目：《图灵之火》报告人：张小平（清华大学计算机系）15:45–16:15报告题目：《IT传奇——微软经典照片背后的计算机往事》报告人：司宏伟（清华大学科学史系）16:15–16:45报告题目：《浅谈机械计算机的复原研究》报告人：王哲然（清华大学科学史系）圆桌会议+闭幕式16:45–17:30主持人：吴国盛（清华大学科学史系）2020-09-22'

In [92]:
res[-1]

[<td height="20" valign="baseline"><div align="center" class="style3">　　简介:</div></td>,
 <td height="20"><p style="line-height: 1.5em; margin-bottom: 10px; margin-top: 10px; text-align: justify; text-indent: 2em;"><img alt="“神机妙算——计算器具历史展”开幕式+论坛 海报_副本.jpg" src="/ueditor/jsp/upload/image/20200922/1600739130083023183.jpg" title="1600739130083023183.jpg"/></p><p style="margin-bottom: 10px; margin-top: 10px; text-align: justify; text-indent: 2em; line-height: 1.75em;"><span style="font-size: 20px; font-family: 宋体, SimSun;"><br/></span></p><p style="margin-bottom: 10px; margin-top: 10px; text-align: justify; text-indent: 2em; line-height: 2em;"><span style="font-size: 20px; font-family: 宋体, SimSun;">清华科学博物馆学论坛是以科学博物馆学和科学仪器史研究为主题的学术活动，由清华大学科学博物馆（筹）和清华大学科学史系主办。论坛以理论探讨为主，与展览实践密切结合，体现高校博物馆特色，促进高校人才培养与大众科学传播。</span></p><p style="margin-bottom: 10px; margin-top: 10px; text-align: justify; text-indent: 2em; line-height: 1.75em;"><span style="font-family: 宋体, SimSun; font-size: 20px; text-indent: 2

In [ ]:

        trs_detail=bs_detail.find_all('tr')
        tr_detail_dict={}
        for tr in trs_detail:
            tds=tr.find_all('td')
            if len(tds)>0:
                node1=tds[0].find('div')
                if node1 is not None and node1.string is not None:
                    s=node1.string.strip()
                    if s in cols:
                        tr_detail_dict[s]=tds[1].string
                    tr_detail_dict['链接']=detail_link
        tag_data_list.append(tr_detail_dict)
#                 #columns: '报告时间','标题','报告人','地点','链接'
#                 cols=tr.find_all('td')
#                 cols[1]=cols[1].find('a')
#                 data_dict[lec_tag].append([col.string for col in cols] + [root+nextpage_url])
nextpage_node=bs_tag.find('a', string=re.compile('.*下.*页.*'))
if nextpage_node is None:
    break
nextpage_url=nextpage_node['href']
tag_page=requests.get(root+nextpage_url)
count+=1

In [41]:
data_dict={}
total_count=0
import time
for tag_root in trs_root:
    start=time.time()
    tag_data_list=[]
    lec_node=tag_root.find('a')
    if lec_node is None:
        break
    lec_tag=lec_node.string
    data_dict[lec_tag]=[]
    lec_url=lec_node['href']
    tag_page=requests.get(root+lec_url)
    count=0
    while True:
        bs_tag=BeautifulSoup(tag_page.text, 'lxml')
        trs_lec=bs_tag.find_all('tr')
        for tr in trs_lec:
            if tr.get('bgcolor', None) is not None and tr.get('class', None) is None:
                detail_url=tr.find_all('td')[1].find('a')['href']
                detail_link=root+detail_url
                detail_page=requests.get(detail_link)
                bs_detail=BeautifulSoup(detail_page.text, 'lxml')
                trs_detail=bs_detail.find_all('tr')
                tr_detail_dict={}
                for tr in trs_detail:
                    tds=tr.find_all('td')
                    if len(tds)>0:
                        node1=tds[0].find('div')
                        if node1 is not None and node1.string is not None:
                            s=node1.string.strip()

                            if s in cols:
                                tr_detail_dict[s]=tds[1].string
                            tr_detail_dict['链接']=detail_link
                tag_data_list.append(tr_detail_dict)
#                 #columns: '报告时间','标题','报告人','地点','链接'
#                 cols=tr.find_all('td')
#                 cols[1]=cols[1].find('a')
#                 data_dict[lec_tag].append([col.string for col in cols] + [root+nextpage_url])
        nextpage_node=bs_tag.find('a', string=re.compile('.*下.*页.*'))
        if nextpage_node is None:
            break
        nextpage_url=nextpage_node['href']
        tag_page=requests.get(root+nextpage_url)
        count+=1
        if count >= 2:
            break
        print(count)
    data_dict[lec_tag]=pd.DataFrame.from_records(tag_data_list)
    end=time.time()
    print(lec_tag, end-start)
    total_count+=1
    if total_count>=2:
        break

KeyboardInterrupt: 

In [62]:
for key in data_dict:
    data_dict[key].to_csv("{}.csv".format(key), index=False)

In [63]:
df=pd.read_csv('经济学.csv')

In [64]:
df.head()

,报告题目:,链接,报告人:,报告时间:,报告地点:,主办单位:
0,讲座预告 | 中美风暴下的风险与对策-金融EMBA师友沙龙,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...,鞠建东教授等,2020-09-12 20:00,线上,清华五道口金融EMBA中心
1,2019中国金融科技学术年会,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...,NaN,2019-12-21 08:30,北京香山饭店,NaN
2,名师微沙龙：文科PI系列第一场,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...,仇军,2019-12-11 16:30,李文正图书馆负一层邺架轩书店,文科建设处
3,清华五道口金融家大讲堂,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...,Sheila M'Mbijjewe,2019-12-10 16:00,清华大学五道口金融学院1-200,清华大学五道口金融学院
4,气候变化大讲堂第14讲：能源低碳转型与气候变化,https://kyybgxx.cic.tsinghua.edu.cn/kybg/xsgg/...,李政,2019-11-28 10:00,清华大学能源与动力工程系系馆报告厅（清华二校门旁）,清华大学气候变化与可持续发展研究院


In [20]:
s='　　简介:'

In [22]:
s.strip()

'简介:'